# Introduction

In this guided project, we're going to practice using our SQL skills to answer business questions.

We'll continue to use the Chinook database that we've used in earlier lessons of this course. The Chinook database is provided as a SQLite database file called chinook.db. 

A copy of the database schema is below:

![Database Schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

# Part 1: Connecting to Datasource & First Looks at the Data

#### Lets connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

#### Let's start by getting familiar with our data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM album
LIMIT 5;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT * FROM employee
LIMIT 2;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


# Part 2: Query that returns each genre, with the number of tracks sold in the USA

- The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce

- The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

- Write a query to find out which genres sell the most tracks in the USA

In [5]:
%%sql
with usa_tracks_sold AS(
    SELECT il.*
    FROM invoice_line il
    INNER JOIN invoice i ON il.invoice_id=i.invoice_id
    INNER JOIN customer c ON i.customer_id=c.customer_id
    WHERE c.country = 'USA')

SELECT 
    g.name GENRE,
    count(usatr.invoice_id) NUMBER_OF_TRACKS_SOLD,
    (count(usatr.invoice_id) / CAST((SELECT COUNT(*) FROM usa_tracks_sold) AS FLOAT))*100.0 PERCENTAGE_SOLD
FROM usa_tracks_sold usatr
INNER JOIN track t ON usatr.track_id = t.track_id
INNER JOIN genre g ON t.genre_id=g.genre_id
GROUP BY 1
ORDER BY 3 desc

Done.


GENRE,NUMBER_OF_TRACKS_SOLD,PERCENTAGE_SOLD
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Based off of percentage sold in USA, Chinook record store would have a better chance selling the Red Tone, Meteor and the Girls, and Slim Jim Bites albums. As well as percentage sold, these genres sell a higher number of tracks compared to Hip Hop/Rap albums, unfortunately.

# Part 3: Query that finds total dollar amount of sales assigned to each sales support agent within company.

- Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

- You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

(Add any extra attributes for that employee that you find are relevant to the analysis.)

In [6]:
%%sql

with sales_agents as (
    SELECT 
        e.first_name||" "||e.last_name AGENT_NAME,
        e.title ROLE,
        e.hire_date HIRE_DATE,
        count(c.customer_id) CUSTOMER_COUNT,
        e.employee_id EMPLOYEE_ID
    FROM employee e
    INNER JOIN customer c ON e.employee_id = c.support_rep_id
    WHERE e.title = 'Sales Support Agent'
    GROUP BY 1
),

invoice_totals_by_customer as (
    SELECT
        c.customer_id CUSTOMER_ID,
        c.support_rep_id SUPPORT_REP_ID,
        CAST(SUM(i.total) AS INT) TOTAL_SALES,
        CAST(AVG(il.unit_price) AS FLOAT) AVG_UNIT_PRICE_PER_CUSTOMER
    FROM invoice i
    INNER JOIN invoice_line il ON i.invoice_id=il.invoice_id
    INNER JOIN customer c ON i.customer_id=c.customer_id
    GROUP BY 1
    ORDER BY 3 DESC
)

SELECT 
    s.agent_name AGENT_NAME,
    s.hire_date HIRE_DATE,
    s.customer_count CUSTOMER_COUNT,
    itc.total_sales TOTAL_SALES,
    itc.avg_unit_price_per_customer AVG_UNIT_PRICE_PER_CUSTOMER
FROM sales_agents s
INNER JOIN invoice_totals_by_customer itc ON 
    s.employee_id=itc.support_rep_id
GROUP BY 1
ORDER BY 4 DESC

Done.


AGENT_NAME,HIRE_DATE,CUSTOMER_COUNT,TOTAL_SALES,AVG_UNIT_PRICE_PER_CUSTOMER
Margaret Park,2017-05-03 00:00:00,20,1682,0.989999999999999
Steve Johnson,2017-10-17 00:00:00,18,1500,0.9899999999999978
Jane Peacock,2017-04-01 00:00:00,21,1433,0.9899999999999981


Margaret Park is the top Sales Support Agent. She able to maximize her interactions with customers , and considereing she started a month after Jane she is relatively very well. 

Steve considering customer to sales ratio, performs the best. He has generated more sales seeing less customers, and started 5 months after Margaret and Jane--fantastic.

Trend as a whole: longer tenure, the better sales are.

# Part 4: Write a query that collates data on purchases from different countries

- Where a country has only one customer, collect them into an "Other" group.

- The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

In [7]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


# PART 5: Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics

- In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. 

- We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [120]:
%%sql
WITH invoice_tracks AS(
    SELECT
        i.invoice_id,
        a.album_id,
        t.track_id
    FROM invoice i 
    INNER JOIN invoice_line il ON i.invoice_id=il.invoice_id
    INNER JOIN track t ON il.track_id=t.track_id
    INNER JOIN album a ON t.album_id=a.album_id
),

albums_in_invoice AS(
    SELECT 
        DISTINCT i.invoice_id,
        COUNT(DISTINCT i.album_id) n_albums_in_invoice
    FROM invoice_tracks i
    GROUP BY 1
    
),

album_tracks AS(
    SELECT
        a.album_id,
        t.track_id 
    FROM album a 
    INNER JOIN track t ON a.album_id=t.album_id
),

album_track_count AS (
    SELECT
        album_id,
        COUNT(DISTINCT track_id) n_of_tracks_in_album
    FROM album_tracks
    GROUP BY 1    
),

invoices_tracks_compare AS(
    SELECT
        i.invoice_id,
        atc.album_id,
        COUNT(DISTINCT i.track_id) n_tracks_invoice,
        atc.n_of_tracks_in_album,
        CASE
            WHEN 
                COUNT(DISTINCT i.track_id) =atc.n_of_tracks_in_album THEN 'full_album_bought'
            ELSE 'single_purchase'
        END 'album_or_single_purchase',
        CASE
            WHEN  
                COUNT(DISTINCT(i.track_id)) = atc.n_of_tracks_in_album THEN 1
            ELSE 0
            END AS full_album_bought_boolean
    FROM invoice_tracks i
    LEFT JOIN album_track_count atc ON i.album_id = atc.album_id
    GROUP BY 1,2
)

SELECT 
    COUNT(DISTINCT invoice_id),
    Album_or_Single_Purchase,
    CAST(COUNT(DISTINCT invoice_id) AS FLOAT) / 
        (SELECT COUNT(*) 
         
         FROM (
        SELECT 
        itc.invoice_id,
        CASE
            WHEN COUNT(album_id) = SUM(full_album_bought_boolean) THEN 'Full album(s)'
            ELSE 'Individual track(s)'
        END 'Album_or_Single_Purchase'
    FROM invoices_tracks_compare itc
    GROUP BY 1
        
        )
        
        
        
        ) Percentage
FROM (
    SELECT 
        itc.invoice_id,
        CASE
            WHEN COUNT(album_id) = SUM(full_album_bought_boolean) THEN 'Full album(s)'
            ELSE 'Individual track(s)'
        END 'Album_or_Single_Purchase'
    FROM invoices_tracks_compare itc
    GROUP BY 1
) AS final_invoice
GROUP BY 2


Done.


COUNT(DISTINCT invoice_id),Album_or_Single_Purchase,Percentage
114,Full album(s),0.18566775244299674
500,Individual track(s),0.8143322475570033


Overwhelmingly, more users purchase indivoidual tracks. So it is advised to follow management's gut decision to only purcahse the individual tracks and not the entire album.